In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install opencv-python
!pip install scikit-video
!pip install scikit-image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 13.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import PIL
import PIL.Image
import skvideo.io
import skimage.transform
import numpy as np
import cv2
from joblib import Parallel, delayed

def process_and_write_image(images,name):
    images = np.array(images).transpose((0,2,3,4,1))
    images = (images + 1)*127.5
    for i in range(images.shape[0]):
        PIL.Image.fromarray(np.around(images[i,0,:,:,:]).astype(np.uint8)).save("./genvideos/" + name + ".jpg")

def read_and_process_video(files,size,nof):
   
    videos = np.zeros((size,nof,64,64,3))
    counter = 0
    for file in files:
        # vcap = cv2.VideoCapture(file) # 0=camera
        # width  = int(vcap.get(cv2.CAP_PROP_FRAME_WIDTH))   # float `width`
        # height = int(vcap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        vid = skvideo.io.vreader(file, num_frames=nof)
        curr_frames = []
        i = 0
        for frame in vid:
            frame = skimage.transform.resize(frame,[64,64])
            curr_frames.append(frame)
            i = i + 1
        curr_frames = np.array(curr_frames)
        curr_frames = curr_frames*255.0
        curr_frames = curr_frames/127.5 - 1
        videos[counter,:,:,:,:] = curr_frames
        counter = counter + 1
    return videos

def process_and_write_video(videos,name):
    videos = np.array(videos)
    videos = np.reshape(videos,[-1,3,32,64,64]).transpose((0,2,3,4,1))
    vidwrite = np.zeros((32,64,64,3))
    for i in range(videos.shape[0]):
        vid = videos[i,:,:,:,:]
        vid = (vid + 1)*127.5
        for j in range(vid.shape[0]):
            frame = vid[j,:,:,:]
            vidwrite[j,:,:,:] = frame
        skvideo.io.vwrite("./genvideos/" +name + ".mp4",vidwrite)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import os
import scipy.misc
import numpy as np
import glob
import sys
from argparse import ArgumentParser
from datetime import datetime
from random import shuffle
from skimage.transform import resize

from tqdm import tqdm
from pickle import load
from matplotlib.pylab import plt
from numpy import arange

epochs = 10
dim = [3,32,64,64]
zd = 100
nb = 32
l = 0.1
c = None
s = 1

class Generator(torch.nn.Module):
    def __init__(self, zdim=zd):
        super(Generator, self).__init__()
        
        self.zdim = zdim
        
        # Background
        self.conv1b = nn.ConvTranspose2d(zdim, 512, [4,4], [1,1])
        self.bn1b = nn.BatchNorm2d(512)

        self.conv2b = nn.ConvTranspose2d(512, 256, [4,4], [2,2], [1,1])
        self.bn2b = nn.BatchNorm2d(256)

        self.conv3b = nn.ConvTranspose2d(256, 128, [4,4], [2,2], [1,1])
        self.bn3b = nn.BatchNorm2d(128)

        self.conv4b = nn.ConvTranspose2d(128, 64, [4,4], [2,2], [1,1])
        self.bn4b = nn.BatchNorm2d(64)

        self.conv5b = nn.ConvTranspose2d(64, 3, [4,4], [2,2], [1,1])

        # Foreground
        self.conv1 = nn.ConvTranspose3d(zdim, 512, [2,4,4], [1,1,1])
        self.bn1 = nn.BatchNorm3d(512)

        self.conv2 = nn.ConvTranspose3d(512, 256, [4,4,4], [2,2,2], [1,1,1])
        self.bn2 = nn.BatchNorm3d(256)

        self.conv3 = nn.ConvTranspose3d(256, 128, [4,4,4], [2,2,2], [1,1,1])
        self.bn3 = nn.BatchNorm3d(128)

        self.conv4 = nn.ConvTranspose3d(128, 64, [4,4,4], [2,2,2], [1,1,1])
        self.bn4 = nn.BatchNorm3d(64)

        self.conv5 = nn.ConvTranspose3d(64, 3, [4,4,4], [2,2,2], [1,1,1])

        # Mask
        self.conv5m = nn.ConvTranspose3d(64, 1, [4,4,4], [2,2,2], [1,1,1])

        # Init weights
        for m in self.modules():
            classname = m.__class__.__name__
            if classname.lower().find('conv') != -1:
                nn.init.normal_(m.weight, mean=0, std=0.01)
                nn.init.constant_(m.bias, 0)
            elif classname.lower().find('bn') != -1:
                nn.init.normal_(m.weight, mean=1, std=0.02)
                nn.init.constant_(m.bias, 0)

    def forward(self, z):
        # Background
        b = F.relu(self.bn1b(self.conv1b(z.unsqueeze(2).unsqueeze(3))))
        b = F.relu(self.bn2b(self.conv2b(b)))
        b = F.relu(self.bn3b(self.conv3b(b)))
        b = F.relu(self.bn4b(self.conv4b(b)))
        b = torch.tanh(self.conv5b(b)).unsqueeze(2)  # b, 3, 1, 64, 64

        # Foreground
        f = F.relu(self.bn1(self.conv1(z.unsqueeze(2).unsqueeze(3).unsqueeze(4))))
        f = F.relu(self.bn2(self.conv2(f)))
        f = F.relu(self.bn3(self.conv3(f)))
        f = F.relu(self.bn4(self.conv4(f)))
        m = torch.sigmoid(self.conv5m(f))   # b, 1, 32, 64, 64
        f = torch.tanh(self.conv5(f))   # b, 3, 32, 64, 64
        
        out = m*f + (1-m)*b

        return out, f, b, m


class Discriminator(torch.nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.conv1 = nn.Conv3d(3, 64, [4,4,4], [2,2,2], [1,1,1])

        self.conv2 = nn.Conv3d(64, 128, [4,4,4], [2,2,2], [1,1,1])
        self.bn2 = nn.BatchNorm3d(128)

        self.conv3 = nn.Conv3d(128, 256, [4,4,4], [2,2,2], [1,1,1])
        self.bn3 = nn.BatchNorm3d(256)

        self.conv4 = nn.Conv3d(256, 512, [4,4,4], [2,2,2], [1,1,1])
        self.bn4 = nn.BatchNorm3d(512)

        self.conv5 = nn.Conv3d(512, 1, [2,4,4], [1,1,1])

        # Init weights
        for m in self.modules():
            classname = m.__class__.__name__
            if classname.lower().find('conv') != -1:
                nn.init.normal_(m.weight, mean=0, std=0.01)
                nn.init.constant_(m.bias, 0)
            elif classname.lower().find('bn') != -1:
                nn.init.normal_(m.weight, mean=1, std=0.02)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = F.leaky_relu(self.conv1(x), 0.2)
        x = F.leaky_relu(self.bn2(self.conv2(x)), 0.2)
        x = F.leaky_relu(self.bn3(self.conv3(x)), 0.2)
        x = F.leaky_relu(self.bn4(self.conv4(x)), 0.2)
        x = self.conv5(x)

        return x

def plot_history(loss, num_epochs, model_type, label):
      plt.plot(np.arange(0, num_epochs, step=1), loss, label=label)
      plt.title(f'HISTORY ({model_type})')
      plt.xlabel('Epochs')
      plt.ylabel('Loss')
      plt.legend(loc='best')
      plt.savefig(model_type+'.png')

In [ ]:
class Discriminator(torch.nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        self.conv1 = nn.Conv3d(3, 64, [4,4,4], [2,2,2], [1,1,1])

        self.conv2 = nn.Conv3d(64, 128, [4,4,4], [2,2,2], [1,1,1])
        self.bn2 = nn.BatchNorm3d(128)

        self.conv3 = nn.Conv3d(128, 256, [4,4,4], [2,2,2], [1,1,1])
        self.bn3 = nn.BatchNorm3d(256)

        self.conv4 = nn.Conv3d(256, 512, [4,4,4], [2,2,2], [1,1,1])
        self.bn4 = nn.BatchNorm3d(512)
        

        self.conv5 = nn.Conv3d(512, 1, [2,4,4], [1,1,1])

        # Init weights
        for m in self.modules():
            classname = m.__class__.__name__
            if classname.lower().find('conv') != -1:
                nn.init.normal_(m.weight, mean=0, std=0.01)
                nn.init.constant_(m.bias, 0)
            elif classname.lower().find('bn') != -1:
                nn.init.normal_(m.weight, mean=1, std=0.02)
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        x = F.leaky_relu(self.conv1(x), 0.2)
        x = F.leaky_relu(self.bn2(self.conv2(x)), 0.2)
        x = F.leaky_relu(self.bn3(self.conv3(x)), 0.2)
        x = F.leaky_relu(self.bn4(self.conv4(x)), 0.2)
        x = self.conv5(x)

In [ ]:
if not os.path.exists("./checkpoints"):
        os.makedirs("./checkpoints")
if not os.path.exists("./genvideos"):
    os.makedirs("./genvideos")


# Model def
G = Generator(zdim=zd).cuda()
D = Discriminator().cuda()

# Optimizer
params_G = list(filter(lambda p: p.requires_grad, G.parameters()))
optimizer_G = optim.Adam(params_G, lr=0.0002, betas=(0.5,0.999))
params_D = list(filter(lambda p: p.requires_grad, D.parameters()))
optimizer_D = optim.Adam(params_D, lr=0.0002, betas=(0.5,0.999))

# Load pretrained
if c is not None:
    G.load_state_dict(torch.load("./checkpoints/{}_G.pth".format(c)).state_dict(), strict=True)
    D.load_state_dict(torch.load("./checkpoints/{}_D.pth".format(c)).state_dict(), strict=True)
    print("Model restored")

# Train
# data file list
data_files = glob.glob("/content/drive/MyDrive/data/*")
print_count = int(len(data_files)/nb)
print(print_count)
gs = []
ds = []

for epoch in tqdm(range(epochs), position=0, leave=False):
    for counter in tqdm(range(print_count), position=1, leave=False):
        batch_files = data_files[counter*nb:(counter+1)*nb]
        video = read_and_process_video(batch_files,nb,32)
        real_video = Variable(torch.from_numpy(video.astype(np.float32).transpose((0,4,1,2,3))), requires_grad=True).cuda()
        # D
        noise = torch.from_numpy(np.random.normal(0, 1, size=[nb,zd]).astype(np.float32)).cuda()
        with torch.no_grad():
            fake_video, f, b, m = G(noise)

        logit_real = D(real_video)
        logit_fake = D(fake_video.detach())

        prob_real = torch.mean(torch.sigmoid(logit_real))
        prob_fake = torch.mean(torch.sigmoid(logit_fake))

        loss_real = F.binary_cross_entropy_with_logits(logit_real, torch.ones_like(logit_real))
        loss_fake = F.binary_cross_entropy_with_logits(logit_fake, torch.zeros_like(logit_fake))
        loss_D = torch.mean(torch.stack([loss_real, loss_fake]))
        ds.append(loss_D.cpu().data.numpy())


        optimizer_D.zero_grad()
        loss_D.backward()
        optimizer_D.step()

        # G
        noise = torch.from_numpy(np.random.normal(0, 1, size=[nb,zd]).astype(np.float32)).cuda()
        gen_video, f, b, m = G(noise)

        logit_gen = D(gen_video)

        loss_gen = F.binary_cross_entropy_with_logits(logit_gen, torch.ones_like(logit_gen))
        loss_G = torch.mean(torch.stack([loss_gen])) + l*F.l1_loss(m, torch.zeros_like(m), True, True)
        gs.append(loss_G.cpu().data.numpy())

        optimizer_G.zero_grad()
        loss_G.backward()
        optimizer_G.step()

        # Print status
        print("Epoch {:d}/{:d} | Iter {:d}/{:d} | D {:.4e} | G {:.4e} | Real {:.2f} | Fake {:.2f}".format(epoch, epochs, counter, print_count, loss_D, loss_G, prob_real, prob_fake))

        process_and_write_video(gen_video[0:1].cpu().data.numpy(), "curr_video")
        process_and_write_image(b.cpu().data.numpy(), "curr_bg")

        if np.mod(counter + 1, 10) == 0 or counter == print_count-1:
            print("Epoch {:d}/{:d} | Iter {:d}/{:d} | D {:.4e} | G {:.4e} | Real {:.2f} | Fake {:.2f}".format(epoch, epochs, counter, print_count, loss_D, loss_G, prob_real, prob_fake))
            process_and_write_video(gen_video[0:1].cpu().data.numpy(), "epoch{}_iter{}_video".format(epoch, counter))
            process_and_write_image(b.cpu().data.numpy(), "epoch{}_iter{}_bg".format(epoch, counter))
            print (".....Writing sample generated videos......")

49


  0%|          | 0/49 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))

  2%|▏         | 1/49 [00:21<16:55, 21.15s/it]

Epoch 0/10 | Iter 0/49 | D 9.6115e-01 | G 1.8463e+01 | Real 0.68 | Fake 0.76
Epoch 0/10 | Iter 1/49 | D 1.0623e+00 | G 1.9658e+00 | Real 0.21 | Fake 0.03



  6%|▌         | 3/49 [01:05<16:52, 22.00s/it]

Epoch 0/10 | Iter 2/49 | D 6.9101e+00 | G 6.8663e+00 | Real 1.00 | Fake 1.00


...

Epoch 6/10 | Iter 11/49 | D 7.8816e-01 | G 3.8147e+00 | Real 0.95 | Fake 0.67



 27%|██▋       | 13/49 [04:25<12:15, 20.44s/it]

Epoch 6/10 | Iter 12/49 | D 1.0607e+00 | G 5.7983e-01 | Real 0.32 | Fake 0.16



 29%|██▊       | 14/49 [04:45<11:55, 20.44s/it]

Epoch 6/10 | Iter 13/49 | D 1.0365e+00 | G 4.8117e+00 | Real 0.91 | Fake 0.81


In [ ]:
plot_history(gs, print_count*epochs, 'VideoGAN', 'Generator Loss')
plot_history(ds, print_count*epochs, 'VideoGAN', 'Discriminator Loss')
print(gs, ds)